In [1]:
from common import *
from config import *
from IPython.display import display
import numpy as np
import pandas as pd

nypd_data = LOAD_DATASET()

# Cleaning data

### Options of populating CMPLNT_FR_DT
1. We just take datetime from CMPLT_TO_DT if difference in hours is not bigger than some threeshold
    - but what if fr_tm and to_tm are the same? Do we really can populate with same date? 
2. Other option we can just populate it with report date 

len(nypd_data[nypd_data['RPT_DT'].isna()]) # 0

len(nypd_data[nypd_data['RPT_DT'] == STR_NULL]) # 0

rpt date is always non null

In [295]:
# First option
mask = nypd_data['CMPLNT_FR_DT'].isna() & nypd_data['CMPLNT_TO_DT'].notnull()

def fill_cmplnt_fr_dt(row, threshold_hours):
    if is_time_difference_within_threshold(row['CMPLNT_FR_TM'], row['CMPLNT_TO_TM'], threshold_hours):
        return row['CMPLNT_TO_DT']
    return row['CMPLNT_FR_DT']

nypd_data.loc[mask, 'CMPLNT_FR_DT'] = nypd_data[mask].apply(fill_cmplnt_fr_dt, threshold_hours=THRESHOLD_HOURS, axis=1)

In [19]:
# Second option
mask = nypd_data['CMPLNT_FR_DT'].isna()
nypd_data.loc[mask, 'CMPLNT_FR_DT'] = nypd_data['RPT_DT']

In [301]:
len(nypd_data[nypd_data['CMPLNT_FR_DT'].isna()])

0

### Cleaning CMPLNT_FR_TM

In [297]:
rows_to_drop = nypd_data[nypd_data['CMPLNT_FR_TM'] == STR_NULL].index

nypd_data.drop(rows_to_drop, inplace=True)

### Cleaning CMPLNT_TO_TM
I dont think we really need those rows, maybe only for plotting, but even then i would just drop them

In [298]:
nypd_data.dropna(subset=['CMPLNT_TO_TM'], inplace=True)

### Filling both to_dt and fr_dt with rpt_dt if they are null

In [299]:
mask = nypd_data['CMPLNT_FR_DT'].isnull() & nypd_data['CMPLNT_TO_DT'].isnull()

nypd_data.loc[mask, 'CMPLNT_FR_DT'] = nypd_data.loc[mask, 'RPT_DT']
nypd_data.loc[mask, 'CMPLNT_TO_DT'] = nypd_data.loc[mask, 'RPT_DT']

In [300]:
null_to_dt = nypd_data[nypd_data["CMPLNT_TO_DT"].isna()]
starting_dt = null_to_dt[null_to_dt['CMPLNT_FR_DT'].isna()]
len(starting_dt)

0

## Cleaning OFNS_DESC

1. First we get all unique codes when OFNS_DESC is null
2. We create a map based on those codes and their OFNS_DESC from other rows
3. We populate original dataset with those values

In [302]:
ky_cd_for_null_offns_desc = nypd_data[nypd_data['OFNS_DESC'] == STR_NULL]['KY_CD'].unique()
ky_cd_to_ofns_desc = nypd_data[nypd_data['OFNS_DESC'] != STR_NULL].set_index('KY_CD')['OFNS_DESC'].to_dict()
nypd_data.loc[nypd_data['OFNS_DESC'] == STR_NULL, 'OFNS_DESC'] = nypd_data['KY_CD'].map(ky_cd_to_ofns_desc)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX


### Cleaning ADDR_PCT_CD 
Maybe we can get this using some external api based on long and lat if needed
currently will just drop them

In [303]:
nypd_data.dropna(subset=['ADDR_PCT_CD'], inplace=True)

### Cleaning of PD_CD

Clearing PD_CD

PD_CD is nan when ky_cd is 101 (ofns_desc is MURDER & NON-NEGL. MANSLAUGHTER	)

My idea is that we create a new pd_cd code and give it the same description as in ofns_desc based on ky_cd (101)

In [305]:
new_code = 976
description = 'MURDER & NON-NEGL. MANSLAUGHTER'
nypd_data.loc[nypd_data['PD_CD'].isna(), 'PD_CD'] = new_code
mask = nypd_data['PD_DESC'] == STR_NULL
nypd_data.loc[mask, 'PD_DESC'] = description

### Cleaning CRM_ATPT_CPTD_CD 

I think we should just drop crimes that were neither completed nor attempted, it is only .0018% of actual dataset

In [306]:
nypd_data.dropna(subset=['CRM_ATPT_CPTD_CD'], inplace=True)
rows_to_drop = nypd_data[nypd_data['CRM_ATPT_CPTD_CD'] == STR_NULL].index

nypd_data.drop(rows_to_drop, inplace=True)

### Cleaning BORO_NM
i think we should just drop those row, it will take great effort to populate this column with data using x/y coordinates
or latitude and longtude.

In [307]:
rows_to_drop = nypd_data[nypd_data['BORO_NM'] == STR_NULL].index

nypd_data.drop(rows_to_drop, inplace=True)

### Cleaning `LOC_OF_OCCUR_DESC` 
I will populate this with `OUTSIDE` value if PARKS_NM is not null
Otherwise with `INSIDE`

In [308]:
# First is outside
mask_parks = nypd_data['PARKS_NM'] != STR_NULL
mask_loc = nypd_data['LOC_OF_OCCUR_DESC'] == STR_NULL
final_mask = mask_parks & mask_loc
nypd_data.loc[final_mask, 'LOC_OF_OCCUR_DESC'] = 'OUTSIDE'

In [309]:
# Everything else with INSIDE
mask = nypd_data['LOC_OF_OCCUR_DESC'] == STR_NULL
mask_nan = nypd_data['LOC_OF_OCCUR_DESC'].isna()
nypd_data.loc[mask, 'LOC_OF_OCCUR_DESC'] = 'INSIDE' 
nypd_data.loc[mask_nan, 'LOC_OF_OCCUR_DESC'] = 'INSIDE' 

### Cleaning `PREM_TYP_DESC`
I would just populate all 39k null values with the most frequent label

In [310]:
most_frequent = nypd_data['PREM_TYP_DESC'].value_counts(dropna=False).idxmax()
mask = nypd_data['PREM_TYP_DESC'] == STR_NULL
nypd_data.loc[mask, 'PREM_TYP_DESC'] = most_frequent

### Cleaning `PARKS_NM`

In [311]:
# I will just replace with unknown that could mean either we really dont know the name or accident occured inside 
mask = nypd_data['PARKS_NM'] == STR_NULL
nypd_data.loc[mask, 'PARKS_NM'] = UNKNOWN

### Cleaning `HADEVELOPT`

In [312]:
mask = nypd_data['HADEVELOPT'] == STR_NULL
nypd_data.loc[mask, 'HADEVELOPT'] = UNKNOWN

### Cleaning `HOUSING_PSA`

In [313]:
mask = nypd_data['HOUSING_PSA'] == STR_NULL
nypd_data.loc[mask, 'HOUSING_PSA'] = UNKNOWN

### Cleaning `SUSP_AGE_GROUP`

In [314]:
all_age_groups = nypd_data['SUSP_AGE_GROUP'].unique()
age_groups_to_change = [item for item in all_age_groups if item not in VALID_AGE_GROUPS]
nypd_data.loc[nypd_data['SUSP_AGE_GROUP'].isin(age_groups_to_change), 'SUSP_AGE_GROUP'] = UNKNOWN
nypd_data['SUSP_AGE_GROUP'].unique()

array(['UNKNOWN', '25-44', '18-24', '45-64', '<18', '65+'], dtype=object)

### Cleaning `SUSP_RACE`

In [315]:
mask = nypd_data['SUSP_RACE'] == STR_NULL
nypd_data.loc[mask, 'SUSP_RACE'] = UNKNOWN

### Cleaning `SUSP_SEX`

In [316]:
mask = nypd_data['SUSP_SEX'] == STR_NULL
nypd_data.loc[mask, 'SUSP_SEX'] = 'U'

### Cleaning PATROL_BORO

maybe we should add some randomness so that it is not alway south/north but randomly changes 

In [317]:
map_boro_to_patrol_boro = {
    'MANHATTAN': 'PATROL BORO MAN SOUTH',
    'BROOKLYN': 'PATROL BORO BKLYN NORTH',
    'BRONX': 'PATROL BORO BRONX',
    'QUEENS': 'PATROL BORO QUEENS NORTH',
    'STATEN ISLAND' : 'PATROL BORO STATEN ISLAND'
}
mask = nypd_data['PATROL_BORO'] == STR_NULL
nypd_data.loc[mask, 'PATROL_BORO'] = nypd_data[mask].apply(
    lambda row: map_boro_to_patrol_boro.get(row['BORO_NM'], row['PATROL_BORO']),
    axis=1
)
nypd_data['PATROL_BORO'].unique()

array(['PATROL BORO MAN SOUTH', 'PATROL BORO BKLYN NORTH',
       'PATROL BORO BRONX', 'PATROL BORO BKLYN SOUTH',
       'PATROL BORO MAN NORTH', 'PATROL BORO QUEENS SOUTH',
       'PATROL BORO QUEENS NORTH', 'PATROL BORO STATEN ISLAND'],
      dtype=object)

### Cleaning `TRANSIT_DISTRICT`
In simple terms it refers to transit areas such as subways and other transportation
We will replace it with 0, meaning crime didn`t occur in transit district

In [318]:
mask = nypd_data['TRANSIT_DISTRICT'].isna()
nypd_data.loc[mask, 'TRANSIT_DISTRICT'] = 0

### Cleaning `STATION_NAME`
I will replace it with Unknown, i dont really believe we will need this feature anyway

In [319]:
mask = nypd_data['STATION_NAME'] == STR_NULL
nypd_data.loc[mask, 'STATION_NAME'] = UNKNOWN

### Cleaning `VIC_AGE_GROUP`

In [320]:
all_age_groups = nypd_data['VIC_AGE_GROUP'].unique()
age_groups_to_change = [item for item in all_age_groups if item not in VALID_AGE_GROUPS]
nypd_data.loc[nypd_data['VIC_AGE_GROUP'].isin(age_groups_to_change), 'VIC_AGE_GROUP'] = UNKNOWN
nypd_data['VIC_AGE_GROUP'].unique()

array(['UNKNOWN', '45-64', '18-24', '65+', '25-44', '<18'], dtype=object)

### Cleaning `VIC_RACE`

In [321]:
mask = nypd_data['VIC_RACE'] == STR_NULL
nypd_data.loc[mask, 'VIC_RACE'] = UNKNOWN

### Cleaning `VIC_SEX`

In [322]:
mask = nypd_data['VIC_SEX'] == STR_NULL
nypd_data.loc[mask, 'VIC_SEX'] = 'U'

Split dataset into some chunks, first is where every value exists, second is some null values but we can replace them by other means
and other is when we have null but no idea how to fill it

In [326]:
nypd_data_nonnull_time = nypd_data[(nypd_data['CMPLNT_TO_DT'].notna()) & (nypd_data['CMPLNT_TO_TM'] != STR_NULL)]

nypd_data_null_time = nypd_data[nypd_data['CMPLNT_TO_DT'].isna() | (nypd_data['CMPLNT_TO_TM'] == STR_NULL)]
nypd_data_null_time = nypd_data_nonnull_time.drop_duplicates()

Checking for null values:

In [327]:
for column in nypd_data_nonnull_time.columns:
    null_count = nypd_data_nonnull_time[column].isnull().sum()
    print(f"Column '{column}' has {null_count} null values.")

Column 'CMPLNT_NUM' has 0 null values.
Column 'CMPLNT_FR_DT' has 0 null values.
Column 'CMPLNT_FR_TM' has 0 null values.
Column 'CMPLNT_TO_DT' has 0 null values.
Column 'CMPLNT_TO_TM' has 0 null values.
Column 'ADDR_PCT_CD' has 0 null values.
Column 'RPT_DT' has 0 null values.
Column 'KY_CD' has 0 null values.
Column 'OFNS_DESC' has 0 null values.
Column 'PD_CD' has 0 null values.
Column 'PD_DESC' has 0 null values.
Column 'CRM_ATPT_CPTD_CD' has 0 null values.
Column 'LAW_CAT_CD' has 0 null values.
Column 'BORO_NM' has 0 null values.
Column 'LOC_OF_OCCUR_DESC' has 0 null values.
Column 'PREM_TYP_DESC' has 0 null values.
Column 'JURIS_DESC' has 0 null values.
Column 'JURISDICTION_CODE' has 0 null values.
Column 'PARKS_NM' has 0 null values.
Column 'HADEVELOPT' has 0 null values.
Column 'HOUSING_PSA' has 0 null values.
Column 'X_COORD_CD' has 0 null values.
Column 'Y_COORD_CD' has 0 null values.
Column 'SUSP_AGE_GROUP' has 0 null values.
Column 'SUSP_RACE' has 0 null values.
Column 'SUSP

In [329]:
for column in nypd_data_nonnull_time.columns:
    null_count = len(nypd_data_nonnull_time[nypd_data_nonnull_time[column] == STR_NULL])
    print(f"Column '{column}' has {null_count} null values.")

Column 'CMPLNT_NUM' has 0 null values.
Column 'CMPLNT_FR_DT' has 0 null values.
Column 'CMPLNT_FR_TM' has 0 null values.
Column 'CMPLNT_TO_DT' has 0 null values.
Column 'CMPLNT_TO_TM' has 0 null values.
Column 'ADDR_PCT_CD' has 0 null values.
Column 'RPT_DT' has 0 null values.
Column 'KY_CD' has 0 null values.
Column 'OFNS_DESC' has 0 null values.
Column 'PD_CD' has 0 null values.
Column 'PD_DESC' has 0 null values.
Column 'CRM_ATPT_CPTD_CD' has 0 null values.
Column 'LAW_CAT_CD' has 0 null values.
Column 'BORO_NM' has 0 null values.
Column 'LOC_OF_OCCUR_DESC' has 0 null values.
Column 'PREM_TYP_DESC' has 0 null values.
Column 'JURIS_DESC' has 0 null values.
Column 'JURISDICTION_CODE' has 0 null values.
Column 'PARKS_NM' has 0 null values.
Column 'HADEVELOPT' has 0 null values.
Column 'HOUSING_PSA' has 0 null values.
Column 'X_COORD_CD' has 0 null values.
Column 'Y_COORD_CD' has 0 null values.
Column 'SUSP_AGE_GROUP' has 0 null values.
Column 'SUSP_RACE' has 0 null values.
Column 'SUSP